In [2]:
! pip install bpemb

     |████████████████████████████████| 1.1 MB 454 kB/s eta 0:00:01
     |████████████████████████████████| 24.2 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 199 kB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 500 kB/s eta 0:00:01
     |████████████████████████████████| 128 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.3 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 526 kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101341 sha256=8ffbb1f490890b0dcc4556f01f4e6b7b08c80681dae7cf62229ecf24671768ba
  Stored in directory: /Users/user/Library/Caches/pip/wheels/bb/1c/9c/412ec03f6d5ac7d41f4b965bde3fc0d1bd201da5ba3e2636de
Successfully built smart-open


In [3]:
from bpemb import BPEmb
from cleantext import clean
from fastai.text import *

# this will download the required model for sub-word tokenization
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

# contruct the vocabulary
itos = dict(enumerate(bpemb_de.words + ['xxpad']))
voc = Vocab(itos)

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


downloading https://nlp.h-its.org/bpemb/de/de.wiki.bpe.vs25000.model


100%|██████████| 664539/664539 [00:01<00:00, 643434.38B/s]


downloading https://nlp.h-its.org/bpemb/de/de.wiki.bpe.vs25000.d300.w2v.bin.tar.gz


100%|██████████| 28012481/28012481 [00:09<00:00, 2873465.23B/s]


In [12]:
data_lm = TextLMDataBunch.from_ids('.', vocab=voc, train_ids=[[1]], valid_ids=[[1]])

In [14]:
config = awd_lstm_lm_config.copy()
config['n_hid'] = 1150

learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5, pretrained=False, config=config)
learn_lm.load('ulmfit_for_german_jfilter')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (1 items)
x: LMTextList
<s>
y: LMLabelList

Path: .;

Valid: LabelList (1 items)
x: LMTextList
<s>
y: LMLabelList

Path: .;

Test: None, model=SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(25001, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(25001, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1150, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1150, 1150, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1150, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=25001, bias=True)
    (output_dp): RNNDropout()
  )
), opt_func=functools.partial(<class 'torch.optim.adam.Adam'>, betas=

In [17]:
learn_lm.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(25001, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(25001, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1150, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1150, 1150, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1150, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=25001, bias=True)
    (output_dp): RNNDropout()
  )
)

In [20]:
learn_lm.predict('berlin ist eine gr')

'berlin ist eine gr m'

In [ ]:
learn_lm = language_model_learner(data_lm)
learn_lm.load('path/to/model/ulmfit_for_german_jfilter.pth')